# Python Code for Extracting the Table and Appendix captions for PCMR Files

### Import Libraries

In [7]:
from io import StringIO
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd
from collections import Counter
import re
import os
import PyPDF2
from PyPDF2 import PdfFileReader
import pandas as pd
import csv
import unicodecsv as csv
import numpy as np
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [8]:
os.chdir('G:/Post Construction/PCMR_All')

### Function for Table Caption Extraction

In [9]:
for dirName, subdirList, fileList in os.walk('G:/Post Construction/PCMR_All'):
    pcmr_pdfs_path = [x for x in fileList if x != 'Thumbs.db']
    #print(pcmr_pdfs_path)
    
#pcmr_pdfs_path = pcmr_pdfs_path

In [10]:
print(len(pcmr_pdfs_path))

246


In [ ]:
page_num = []
table_caption = []
table_caption_next = []
#table_caption_next_next = []
file_name = []
exceptions_list = ['..', 'Table of Content', 'TABLE OF CONTENTS', 'Table des matières', 'TABLE DES MATIÈRES']
last_ptag_starts_with_key = 0
#last_last_ptag_starts_with_key = 0
for a, x in enumerate(pcmr_pdfs_path):
    data = parser.from_file(x, xmlContent=True)
    try:
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})
        for b, p in enumerate(pages):
            pages_text = [x.text for x in p.find_all('p')]
            for y in pages_text:
                if last_ptag_starts_with_key == 1:
                    table_caption_next.append(y.replace('\n',' '))
                last_ptag_starts_with_key = 0
        #             elif last_last_ptag_starts_with_key == 1:
        #                 table_caption_next_next.append(y.replace('\n',' '))
        #        last_last_ptag_starts_with_key = 0

                y = y.strip()
                if y.startswith(('Table', 'Appendix', 'APPENDIX', 'TABLE')) and not any(substring in y for substring in exceptions_list):
                    #y.split(' ')
                    page_num.append(b + 1)
                    table_caption.append(y)
                    file_name.append(x)
                    last_ptag_starts_with_key = 1
                        #last_last_ptag_starts_with_key = 1
    except:
        print('File {} unable to be parsed using Tika (Error 500)'. format(x))
print(table_caption)

In [5]:
page_num = []
table_caption = []
table_caption_next = []
#table_caption_next_next = []
file_name = []
exceptions_list = ['..', 'Table of Content', 'TABLE OF CONTENTS', 'Table des matières', 'TABLE DES MATIÈRES']
last_ptag_starts_with_key = 0
#last_last_ptag_starts_with_key = 0
for a, x in enumerate(pcmr_pdfs_path):
    data = parser.from_file(x, xmlContent=True)
    try:
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})
        for b, p in enumerate(pages):
            pages_text = [x.text for x in p.find_all('p')]
            for y in pages_text:
                if last_ptag_starts_with_key == 1:
                    table_caption_next.append(y.replace('\n',' '))
                last_ptag_starts_with_key = 0
        #             elif last_last_ptag_starts_with_key == 1:
        #                 table_caption_next_next.append(y.replace('\n',' '))
        #        last_last_ptag_starts_with_key = 0

                ytest = y.replace('\n','').replace('\r','')
                ytest = ytest.split(" ")
                ytest_no_blanks = ""
                for ytes in ytest:
                    if len(ytes) > 2:
                        ytest_no_blanks = ytest_no_blanks + ytes + " "
                        
                if y.startswith(('Table', 'Appendix', 'APPENDIX', 'TABLE')) and not any(substring in y for substring in exceptions_list):
                    page_num.append(b + 1)
                    table_caption.append(y.replace('\n','').replace('\r',''))
                    file_name.append(x)
                    last_ptag_starts_with_key = 1
                        #last_last_ptag_starts_with_key = 1
    except:
        print('File {} unable to be parsed using Tika (Error 500)'. format(x))
print(table_caption)

2020-03-03 21:39:54,309 [MainThread  ] [WARNI]  Tika server returned status: 500


File A1X4H8 - Post Construction Environmental Monitoring Report - Year 1 - Spread 8 - Alberta Clipper Project.pdf unable to be parsed using Tika (Error 500)
['Appendix A Figures ', 'Appendix B Tables 1 and 2 - Watino Crossover and Calais Extension - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List ', 'Table 1: Watino Crossover - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List ', 'Table 1: Watino Crossover - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List ', 'Table 2: Calais Extension - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List ', 'Table 2: Calais Extension - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List ', 'Appendix C Photos ', 'APPENDIX A  ', 'APPENDIX B  ', 'APPENDIX C  ', 'Table 1: Setting – Cushing Expansion Pump Stations ', 'Table 2 in this report identifies specific areas related to condition compliance and the action plan implemented to address and complete the commitments noted in the March 3, 2011 compliance letter. The 

In [6]:
print(len(file_name),
len(table_caption),
len(page_num),
len(table_caption_next))

5243 5243 5243 5243


In [8]:
def categorize_tables(list_titles):
    categories = []
    for title in table_caption:
        title_string = title.replace('\n','').replace('–',"").replace(":","").replace("-","").replace("(","")
        title_string = title_string.replace('.','')
        category = 0
        
        title_words = re.split(" ", title_string)
        
        title_words_no_blanks = []
        for title_word in title_words:
            if len(title_word) > 2:
                title_words_no_blanks.append(title_word)
            elif len(title_word) > 0:
                if title_word[0].isdigit():
                    title_words_no_blanks.append(title_word)
                    
        category = 0
    
        if len(title_words_no_blanks) > 2:
            if(title_words_no_blanks[2][0].isupper() or title_words_no_blanks[2][0].isdigit()):
                category = 1 
            if(title_words_no_blanks[1][-1]== ',') or (title_words_no_blanks[1][-1]== ']') or (title_words_no_blanks[1][-1]== ')'):
                category = 0
            if(title_words_no_blanks[2][:4].lower() == 'cont'):
                category = 0.35
            
            
        if len(title_words_no_blanks) == 3 :
            category = 0.5
            if(title_words_no_blanks[1][0].isdigit() and  title_words_no_blanks[1][0].isupper()):
                category = 0.65
            if(title_words_no_blanks[2][0].lower() == 'c'):
                category = 0.55
            if(title_words_no_blanks[2][:4].lower() == 'cont'):
                category = 0.35
        
        if len(title_words_no_blanks) < 3: 
            category = 0.75
        
        categories.append(category)
        print("{} is {}".format(title, category))
        
    return(categories)

In [9]:
categories = categorize_tables(table_caption)

Appendix A Figures  is 0.75
Appendix B Tables 1 and 2 - Watino Crossover and Calais Extension - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List  is 1
Table 1: Watino Crossover - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List  is 1
Table 1: Watino Crossover - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List  is 1
Table 2: Calais Extension - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List  is 1
Table 2: Calais Extension - Summary of 2013 PCRM Assessment and 2012 PCMP Issues List  is 1
Appendix C Photos  is 0.75
APPENDIX A   is 0.75
APPENDIX B   is 0.75
APPENDIX C   is 0.75
Table 1: Setting – Cushing Expansion Pump Stations  is 1
Table 2 in this report identifies specific areas related to condition compliance and the action plan implemented to address and complete the commitments noted in the March 3, 2011 compliance letter. The table also addresses Years 1, 2 and 3 post-construction monitoring observations and the completion status for those observ

In [10]:
print(len(categories))

5243


In [11]:
final_table_caption = []
for i in range(len(categories)):
    if categories[i] > 0.49 and categories[i] < 1:
        final_table_caption.append(table_caption[i].strip() + " " + table_caption_next[i].strip())
    else:
        final_table_caption.append(table_caption[i].strip())
len(final_table_caption)

5243

In [12]:
for i in range(len(categories)):
    if categories[i] == 0.35:
        if(file_name[i]== file_name[i-1]):
            final_table_caption[i] = final_table_caption[i-1]

### Create a dataframe with columns as actual file name, page number where table/image was captured followed by table/image caption

In [13]:
lists = zip(file_name, page_num, table_caption, table_caption_next, final_table_caption, categories)
columns = ['file_name', 'page_number', 'table_title', 'table_title_next', 'final_table_caption', 'categories']
df = pd.DataFrame(lists, columns = columns)

In [14]:
df.loc[df['final_table_caption'].str.startswith('table'), 'Category'] = 'Table'
df.loc[df['final_table_caption'].str.startswith('appendix'), 'Category'] = 'Appendix'
df.loc[df['final_table_caption'].str.startswith('Table'), 'Category'] = 'Table'
df.loc[df['final_table_caption'].str.startswith('Appendix'), 'Category'] = 'Appendix'
df.loc[df['final_table_caption'].str.startswith('TABLE'), 'Category'] = 'Table'
df.loc[df['final_table_caption'].str.startswith('APPENDIX'), 'Category'] = 'Appendix'
df.head(50)
len(df)

5243

Source: https://stackoverflow.com/questions/4389644/regex-to-match-string-containing-two-names-in-any-order

In [204]:
'''new_df = df[(~df['final_table_caption'].str.contains(r'(?=.*environmental)(?=.*issues)', regex = True, case = False)) & (df['Category'] == 'Appendix')].index
df.drop(new_df, inplace = True)
df.head(50)
len(df)'''

"new_df = df[(~df['final_table_caption'].str.contains(r'(?=.*environmental)(?=.*issues)', regex = True, case = False)) & (df['Category'] == 'Appendix')].index\ndf.drop(new_df, inplace = True)\ndf.head(50)\nlen(df)"

In [16]:
df = df[~df['table_title'].str.startswith('TABLES')]
len(df)

5240

In [17]:
df = df[~df['table_title'].str.startswith('Tables')]
len(df)

5183

In [18]:
df = df[~df['table_title'].str.startswith('TABLE OF CONTENTS')]
len(df)

5170

In [20]:
df['filename_finalcaption'] = df['file_name'] + df['final_table_caption']
len(df)

5170

In [21]:
fuzz_ratio = []
file_final_caption = df['filename_finalcaption'].to_list()
for i in range(len(file_final_caption)):
    fuzz_ratio.append(fuzz.WRatio(file_final_caption[i], file_final_caption[i-1]))
len(fuzz_ratio)

5170

In [22]:
df['fuzzy_wuzzy_similarity'] = fuzz_ratio
df = df.reset_index(drop=True)
len(df)
df.head()

,file_name,page_number,table_title,table_title_next,final_table_caption,categories,Category,filename_finalcaption,fuzzy_wuzzy_similarity
0,2013-12-10- Watino and Calais Second PCMP Repo...,9,Appendix A Figures,,Appendix A Figures,0.75,Appendix,2013-12-10- Watino and Calais Second PCMP Repo...,56
1,2013-12-10- Watino and Calais Second PCMP Repo...,14,Appendix B Tables 1 and 2 - Watino Crossover a...,,Appendix B Tables 1 and 2 - Watino Crossover a...,1.00,Appendix,2013-12-10- Watino and Calais Second PCMP Repo...,86
2,2013-12-10- Watino and Calais Second PCMP Repo...,15,Table 1: Watino Crossover - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 1: Watino Crossover - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,91
3,2013-12-10- Watino and Calais Second PCMP Repo...,16,Table 1: Watino Crossover - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 1: Watino Crossover - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,100
4,2013-12-10- Watino and Calais Second PCMP Repo...,17,Table 2: Calais Extension - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 2: Calais Extension - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,90


In [23]:
df['Contains_Target_Words'] = 'No'
df['Contains_Target_Words'][df['final_table_caption'].str.contains(r'(?=.*list)(?=.*issues)|(?=.*lists)(?=.*issue)|(?=.*lists)(?=.*issues)|(?=.*lists)(?=.*issue(s))|(?=.*list)(?=.*concerns)|(?=.*lists)(?=.*concern)|(?=.*lists)(?=.*concerns)|(?=.*lists)(?=.*concern(s))|(?=.*summary)(?=.*issues)|(?=.*summary)(?=.*issue)|(?=.*summary)(?=.*issue(s))|(?=.*summary)(?=.*concerns)|(?=.*summary)(?=.*concern)|(?=.*summary)(?=.*concern(s))|(?=.*issue)|(?=.*issues)|(?=.*issue(s))|(?=.*summary)|(?=.*summaries)|(?=.*results)|(?=.*deficiencies)|(?=.*recommendations)|(?=.*recommendation)|(?=.*tracking)|(?=.*recommendations)|(?=.*concern)|(?=.*concerns)', regex = True, case = False)] = 'Yes'

C:\Anaconda\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
df.head()

,file_name,page_number,table_title,table_title_next,final_table_caption,categories,Category,filename_finalcaption,fuzzy_wuzzy_similarity,Contains_Target_Words
0,2013-12-10- Watino and Calais Second PCMP Repo...,9,Appendix A Figures,,Appendix A Figures,0.75,Appendix,2013-12-10- Watino and Calais Second PCMP Repo...,56,No
1,2013-12-10- Watino and Calais Second PCMP Repo...,14,Appendix B Tables 1 and 2 - Watino Crossover a...,,Appendix B Tables 1 and 2 - Watino Crossover a...,1.00,Appendix,2013-12-10- Watino and Calais Second PCMP Repo...,86,Yes
2,2013-12-10- Watino and Calais Second PCMP Repo...,15,Table 1: Watino Crossover - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 1: Watino Crossover - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,91,Yes
3,2013-12-10- Watino and Calais Second PCMP Repo...,16,Table 1: Watino Crossover - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 1: Watino Crossover - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,100,Yes
4,2013-12-10- Watino and Calais Second PCMP Repo...,17,Table 2: Calais Extension - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 2: Calais Extension - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,90,Yes


In [26]:
num_pages=[]
for index, row in df.iterrows():
    pdfFileObj = open(row['file_name'],'rb')
    pdfReader = PyPDF2.PdfFileReader(row['file_name'])
    try:    
        num_pages_pdf = pdfReader.getNumPages()
        num_pages.append(num_pages_pdf)
    except Exception as e:
        print('{}'.format(row['file_name']),str(e))
        num_pages.append('')

A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H1 - Murphy Tupper Pipeline PCMP - 2009 Report Body.pdf File has not been decrypted
A1R2H4 - Murphy Tupper Pipeline PCMP - 2009 Report Appendicies II to V.pdf File has not been decrypted
A1R2H4 - Murphy Tupper Pipeline PCMP - 2009 Report Appendicies II to V.pdf File has not been decrypted
A1R2H4 - Murphy Tupper Pipeline PCMP - 2009 Report Appendicies II to V.pdf File has not be

In [27]:
len(num_pages)

5170

In [28]:
df['num_pages'] = num_pages
df = df.reset_index(drop=True)
len(df)

5170

In [29]:
df.head()

,file_name,page_number,table_title,table_title_next,final_table_caption,categories,Category,filename_finalcaption,fuzzy_wuzzy_similarity,Contains_Target_Words,num_pages
0,2013-12-10- Watino and Calais Second PCMP Repo...,9,Appendix A Figures,,Appendix A Figures,0.75,Appendix,2013-12-10- Watino and Calais Second PCMP Repo...,56,No,22
1,2013-12-10- Watino and Calais Second PCMP Repo...,14,Appendix B Tables 1 and 2 - Watino Crossover a...,,Appendix B Tables 1 and 2 - Watino Crossover a...,1.00,Appendix,2013-12-10- Watino and Calais Second PCMP Repo...,86,Yes,22
2,2013-12-10- Watino and Calais Second PCMP Repo...,15,Table 1: Watino Crossover - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 1: Watino Crossover - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,91,Yes,22
3,2013-12-10- Watino and Calais Second PCMP Repo...,16,Table 1: Watino Crossover - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 1: Watino Crossover - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,100,Yes,22
4,2013-12-10- Watino and Calais Second PCMP Repo...,17,Table 2: Calais Extension - Summary of 2013 PC...,Legal Location Land Use Recommendation(s),Table 2: Calais Extension - Summary of 2013 PC...,1.00,Table,2013-12-10- Watino and Calais Second PCMP Repo...,90,Yes,22


Source: https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s

In [37]:
df1 = pd.read_csv('G:/Post Construction/metadata csvs/metadata_pcmr_Feb19.csv', encoding= 'unicode_escape', usecols = ['file_name','URL','DataID_pdf','Project_Name'])

In [48]:
new_df = df.merge(df1, on='file_name', how='left')

In [60]:
df2 = pd.read_csv('G:/Post Construction/Analysis Excel csv Files/metadata_table_count.csv', encoding= 'unicode_escape', usecols = ['file_name', 'num_tables_camelot'])

In [61]:
df3 = new_df.merge(df2, on='file_name', how='left')

In [65]:
df3.to_csv('table_appendix_titles_Mar05.csv', encoding = 'utf-8-sig')

In [233]:
###lookup_dict = df.to_dict('records')

### Get Frequency Count of words 'resolved' and 'unresolved' occuring on pages where tables were found

In [29]:
def count_word_in_text(page_content, key):
    page_content = page_content.lower()
    words = page_content.split()
    count = 0
    for word in words:
        if key in word:
            count = count +1       
    return(count)

In [30]:
resolved_count = []
unresolved_count = []
for index, row in df.iterrows():
    pdfFileObj = open(row['file_name'],'rb')
    pdfReader = PyPDF2.PdfFileReader(row['file_name'])
    try:    
        page = pdfReader.getPage(row['page_number']-1)
        page_content = page.extractText()
        unresol_count = count_word_in_text(page_content, "unresolved")
        resol_count = count_word_in_text(page_content, "resolved")- unresol_count
        #print("File Name: {}, Page Number: {}, Resolved Count: {}, Unresolved Count: {}".format(row['file_name'], row['page_number'], resol_count, unresol_count))
        resolved_count.append(resol_count)
        unresolved_count.append(unresol_count)
    except Exception as e:
        print('{}'.format(row['file_name']),str(e))

In [31]:
df['Resolved_count'] = resolved_count
df['Unresolved_count'] = unresolved_count

In [32]:
df.head(50)
df.to_csv('Table_appendix_captions.csv', encoding = 'utf-8-sig')

In [122]:
last_table_name = "Initializer_dummy_table_caption"
last_file_name = "Initializer_dummy_file_name"
last_page_no = 0
last_end_page = 0
start_page = 0
end_page = 0
last_match = 0
resolved_count = 0
unresolved_count = 0

last_resolved_count = 0
last_unresolved_count = 0

table_names = []
start_pages = []
end_pages = []
resolved_counts = []
unresolved_counts = []
fileNames = []

fileNames.append(last_file_name)
table_names.append(last_table_name)
start_pages.append(last_page_no)
#resolved_counts.append(0)
#unresolved_counts.append(0)

for index, row in df.iterrows():
    if(row['page_number'] == last_page_no + 1 and row['final_table_caption'][:10] == last_table_name):
        if last_match == 0:
            #resolved_counts.append(resolved_count)
            #unresolved_counts.append(unresolved_count)
            resolved_count = last_resolved_count
            unresolved_count = last_unresolved_count
        
        
        resolved_count = resolved_count + row['Resolved_count']
        unresolved_count = unresolved_count + row['Unresolved_count'] 
        end_page = row['page_number']
        last_match = 1
        
    elif last_match == 1:
        start_page = row['page_number']
        if last_match == 1:
            end_pages.append(end_page)
        else:
            end_pages.append(last_end_page)
            
        last_end_page = start_page
        last_match = 0
        
        table_names.append(row['final_table_caption'])
        start_page = row['page_number']
        start_pages.append(row['page_number'])
        fileNames.append(row['file_name'])
        
        last_resolved_count = row['Resolved_count']
        resolved_count = resolved_count + row['Resolved_count']
        
        last_unresolved_count = row['Unresolved_count']
        unresolved_count = unresolved_count + row['Unresolved_count']
        
        resolved_counts.append(resolved_count)
        unresolved_counts.append(unresolved_count)
        resolved_count = 0        
        unresolved_count = 0
        
        
        
    last_table_name = row['final_table_caption'][:10]
    last_page_no = row['page_number']
        
if last_match == 1:
    end_pages.append(end_page)
    end_pages.append(last_end_page)
    #else:
    #end_pages.append(end_page)
            
    last_end_page = start_page
    last_match = 0
        
    table_names.append(row['final_table_caption'])
    start_page = row['page_number']
    start_pages.append(row['page_number'])
    fileNames.append(row['file_name'])
        
    resolved_count = resolved_count + row['Resolved_count']
    resolved_counts.append(resolved_count)
    resolved_count = 0
    
    unresolved_count = unresolved_count + row['Unresolved_count']
    unresolved_counts.append(unresolved_count)
    unresolved_count = 0
    
else:
    end_pages.append(last_page_no)
    resolved_counts.append(resolved_count)
    unresolved_counts.append(unresolved_count)

In [124]:
df_combined = pd.DataFrame({'file_name': fileNames,
                            'final_table_names' : table_names,
                         'start_page_number' : start_pages,
                         'end_page_number' : end_pages,
                           'resolved_counts': resolved_counts,
                           'unresolved_counts': unresolved_counts})
df_combined.head(15)

,file_name,final_table_names,start_page_number,end_page_number,resolved_counts,unresolved_counts
0,Initializer_dummy_file_name,Initializer_dummy_table_caption,0,0,0,0
1,2013-12-10- Watino and Calais Second PCMP Repo...,Table 1: Watino Crossover - Summary of 2013 PC...,15,16,0,0
2,2013-12-10- Watino and Calais Second PCMP Repo...,Table 2: Calais Extension - Summary of 2013 PC...,17,18,4,3
3,2014 Post Construction Environmental Monitorin...,APPENDIX B ENVIRONMENTAL ISSUES LIST FROM THE ...,21,21,0,0
4,2014-01-28 Keystone PCEM Year 3 - A3T3I4.pdf,Table 1: Setting – Cushing Expansion Pump Stat...,8,8,0,0
5,2014-01-28 Keystone PCEM Year 3 - A3T3I4.pdf,Table 2: 2013 Status and Areas \nNew Pump Stat...,11,12,0,0
6,2014-01-28 Keystone PCEM Year 4 - A3T3H0.pdf,Table 1: Status of Areas of Subsoil Compaction...,13,13,0,0
7,2014-01-28 Keystone PCEM Year 4 - A3T3H0.pdf,Table 2: Keystone Long-Term Monitoring Plots,17,17,0,0
8,2014-01-28 Keystone PCEM Year 4 - A3T3H0.pdf,Table 3: List of Environmental Deficiencies,24,24,0,1
9,2014-01-28 Keystone PCEM Year 4 - A3T3H0.pdf,Table 4: Status of Issues and Concerns \nFor N...,29,31,0,4


In [119]:
df_combined.to_csv('test.csv', index = False, encoding = 'utf-8-sig')

### Consultant Name and Monitoring Year Analysis
After running first four chunks of code, start from this part of the python file to extract the consultant names and monitoring year

#### Create Dataframe to capture Data

In [12]:
columns = ['pcmr_file_Name', 'consultant_names'] #required columns
metadata = pd.DataFrame(columns = columns) #create dataframe

In [15]:
pdf_dict = {}
pdf_files = []
text_strings = []
companies_all = []
for file in pcmr_pdfs_path:
    if file.endswith('.pdf'):
        pdf_files.append(file)
        print(file)
        pdfFileObj = open(file,'rb') ##creating a pdf file object
        pdfReader = PyPDF2.PdfFileReader(file)  ##creating pdf reader object
        number_of_pages = pdfReader.getNumPages()
        for page_number in range(number_of_pages):
            page = pdfReader.getPage(page_number)
            page_content = page.extractText()
            pdf_dict[file] = page_content #Add to dictionary with key = filename
            text_strings.append(page_content)

2013-12-10- Watino and Calais Second PCMP Report - A3R8A7.pdf
2014 Post Construction Environmental Monitoring Report Ref. Order XG-A083-05-2008 - A4G9K4.pdf
2014-01-28 Keystone PCEM Year 3  - A3T3I4.pdf
2014-01-28 Keystone PCEM Year 4  - A3T3H0.pdf
2014-01-31- Cheecham Kettle River First PCMP Report Part 1 - A3T5F2.pdf
6873_Year 1 PCEM Pembina Beatton River - A3T4K3.pdf
A0L2F6 - 002 Ekwan Post Construction Monitoring - Sept. 04.pdf
A0L3K4 - Kwoen Facilities and Re-injection Extension Pipeline Post-Construction Environmental Report.pdf
A0T5H7 - Kwoen Re-injection Pipeline - 3rd Year Post-Construction Environmental Report.pdf
A1C1E6 - 2007 PCM Report Year 1.pdf
A1C1E9 - 2007 PCM Report Year 1.pdf
A1D2H0 - Stittsville Post-Construction Monitoring Report - Part 1 of 3.pdf
A1H6G5 - Post-Construction Monitoring Report Year 1.pdf
A1H6G8 - Post-Construction Monitoring Report Year 2.pdf
A1H9Y3 - Year One Post-Construction Environmental Monitoring Report.pdf
A1I1H3 - PCM Report Year 1.pdf
A1I1H6

KeyError: '/Contents'

In [ ]:
num_pages=[]
for index, row in df.iterrows():
    pdfFileObj = open(row['file_name'],'rb')
    pdfReader = PyPDF2.PdfFileReader(row['file_name'])
    try:    
        num_pages_pdf = pdfReader.getNumPages()
        num_pages.append(num_pages_pdf)
    except Exception as e:
        print('{}'.format(row['file_name']),str(e))
        num_pages.append('')